In [1]:
import glob #to read the files
import json #to work if .json
import numpy as np #to math
import pandas as pd #to save the data
import math #to convertion calculus
from astropy.time import Time #to time converting
from astropy import units #time correction
from astropy.coordinates import SkyCoord #time correction
from scipy import interpolate #to interpolate the wavelength and flux
from tabulate import tabulate #to export in table format
from scipy.interpolate import interp1d
#import scipy.optimize as opt
from scipy import optimize
from scipy import signal
import os.path
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy import stats
import os.path 
import sncosmo

In [2]:
kernel = 10.0 * RBF(length_scale=10.0, length_scale_bounds=(5, 15)) + WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e+1))

In [3]:
csp = sncosmo.get_magsystem('csp')
ab = sncosmo.get_magsystem('ab')
vega = sncosmo.get_magsystem('vega')

In [4]:
%run ./functions.ipynb

In [5]:
sn_names=glob.glob("*.json")

In [6]:
maxvalues = []

In [7]:
list_data = []
lightc = []

In [8]:
for j in range(0,len(sn_names)):  
    
    name = sn_names[j]

    print(name)
    
    with open(name, "r") as read_file:
        data = json.load(read_file)

    url = name
    if url.endswith('.json'):
        url = url[:-5]

    
    redshift = redshift_value(data[url])
    
    if redshift is not None:

        t0 = max_time(data[url])

        mag = []
        time = []
        system = []

        for i in range(0,len(data[url]["photometry"])):

            if "band" in data[url]["photometry"][i]:

                if data[url]["photometry"][i]["band"] == "V":
                    
                    if "system" in data[url]["photometry"][i]:
                        
                        system.append(data[url]["photometry"][i]["system"])

                        mag.append(float(data[url]["photometry"][i]["magnitude"]))
                        time.append(float(data[url]["photometry"][i]["time"]))
                    
                    else:
                        
                        system.append("w")

                        mag.append(float(data[url]["photometry"][i]["magnitude"]))
                        time.append(float(data[url]["photometry"][i]["time"]))
        
        
        if len(time) > 3:
    
            #maxvalues.append(min(mag))
            
            ntime = []
            nmag = []
            for i in range(0,len(mag)):

                if np.abs(time[i]- min(time)) < 100:
                        
                    if system[i] == "CSP":
                        
                        
                        nmag.append(csp.band_mag_to_flux(mag[i], 'cspb')*(1+redshift)**3)
                        ntime.append([time[i]])
                        
                    if system[i] == "AB":
                        #print("ab")
                        
                        nmag.append(ab.band_mag_to_flux(mag[i], 'cspb')*(1+redshift)**3)
                        ntime.append([time[i]])
                        
                    if system[i] == "Vega":
                        
                        #print("vega")
                        nmag.append(vega.band_mag_to_flux(mag[i], 'cspb')*(1+redshift)**3)
                        ntime.append([time[i]])   
                        
                    if system[i] == "w":
                        
                        #print("w")
                        nmag.append(vega.band_mag_to_flux(mag[i], 'cspb')*(1+redshift)**3)
                        ntime.append([time[i]])                     
                                      
                    #print(np.abs(time[i][0] - min(time[i])))                         
                
            if len(nmag) > 4 :   
                
                gp = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(ntime, nmag)

                X_ = np.linspace(min(ntime)[0], max(ntime)[0], 1000)

                y_mean = gp.predict(X_[:, np.newaxis])

                y_mean = list(y_mean)

                idx = y_mean.index(max(y_mean))

                if idx > 1: 
                    
                    maxvalues.append([max(y_mean), sn_names[j]])

                    lightc.append([X_, y_mean])                    

                #list_data.append(max(y_mean))
                #list_data.append([X_, y_mean])

SN2008bf.json
0.024027
SN2007ca.json
0.014066
SN2007sw.json
0.0252
SN2006hb.json
0.0153328042114802
SN2006gj.json
0.02829
SN2006gz.json
None
SN2005am.json
0.007899
SN2006et.json
0.02217798925424974
SN2002ck.json
0.029864
SN1998es.json
0.01057
SN2008ec.json
0.016317
SN1995bd.json
None
SN2012dn.json
0.010187
SN2013cs.json
0.009
SN1995E.json
0.01158
SN2002cd.json
0.010344
SN2001bp.json
0.094859
SN2002bo.json
0.00424
SN2008fp.json
0.005656012429203772
SN1992A.json
0.00626
SN2007ap.json
0.015818
SN2001N.json
0.021025
SN2008ae.json
0.03006
SN2007bm.json
0.006211
SN1999cp.json
0.00948
SN2007F.json
0.023591
SN2005hf.json
0.04311
SN2003W.json
0.020071
SN2006bt.json
0.032156
SN2012ht.json
0.003559
SN2006kf.json
0.02129693315973835
SN2012hd.json
0.012
SN2007bj.json
0.016708
SN2007co.json
0.026962
LSQ12gdj.json
0.03
SN1995ak.json
0.02272
SN1994T.json
0.03466
SN1997cn.json
0.0162
SN1994S.json
0.01518
SN2014J.json
0.000841
SN2008hv.json
0.012526589416171241
SN2014at.json
0.035
SN2005kc.json
0.01512


SN2008dr.json
0.041449
SN2001el.json
0.00536
SN2007hj.json
0.01411
SN2008J.json
0.015874
SN2003du.json
0.006381
SN2000dn.json
0.032065
SN2004ey.json
0.01579444768273275
SN2005eq.json
0.028974294942283096
SN2006oa.json
redshift nao encontrado
SN2006ev.json
0.02873
SN2007ax.json
0.006861
SN2008ar.json
0.026147
SN1997dt.json
0.007318
SN2007ai.json
0.031629058849925284
SN1998co.json
0.01807
SN2002hw.json
0.017535
SN2006cf.json
0.041549
SN2005hk.json
0.01306
SN2004gs.json
0.02660272499077032
SN1995ac.json
0.0499
SN2002ar.json
0.03009
SN1998V.json
0.01759
SN1992G.json
0.00529
SN2003it.json
0.025121
SN2005ls.json
0.021118
SN2009ag.json
0.00864
SN2002aw.json
0.026145
SN2006X.json
0.00524
SN2005el.json
0.01490009610926979
SN1991M.json
0.007235
SN2002av.json
0.049041
SN1983N.json
0.00321
SN2004S.json
0.00937
SN1989B.json
0.002425
SN2005bo.json
0.013870133763326464
SN2005eo.json
None
SN1997do.json
0.01012
SN1998dx.json
0.054221
SN2006te.json
0.031592
SN1998bp.json
0.01043
SN2006ax.json
0.016725
S

0.022109
SN2006mr.json
0.00586826659018258
SN2000cf.json
0.0438
SN2007au.json
0.020584
SN2001br.json
0.020628
SN2013ew.json
0.056
SN1999aa.json
0.01444
SN2005cf.json
0.006461
SN2004at.json
0.023131
SN2005ao.json
0.038407
LSQ12fhs.json
0.033145
SN2006cz.json
0.041799
SN1995D.json
0.006561
SN2002eu.json
0.037626
SN1998ab.json
None
SN2006D.json
0.008529
SN1989M.json
0.00507
SN2003aa.json
None
SN1998aq.json
0.003699
SN2003eh.json
None
SN2005A.json
0.01914
SN2007on.json
0.006494
SN2000fa.json
0.02127
SN2000B.json
0.01983
SN2003ep.json
redshift nao encontrado
SN1983G.json
0.00618
SN2003M.json
0.02424
SN1956A.json
0.00559
SN2003kf.json


0.007388
SN2001bg.json
0.007122
SN2008s1.json
0.0221
SN2001cp.json
0.022402
LSQ12gpw.json
0.058
SN2001fe.json
0.013539
SN2005gj.json
0.0616
SN2001C.json
0.0109
SN2007nq.json
0.045049248780426154
SN2005eu.json
0.034901
SN1999ac.json
0.0115
SN2006le.json
0.017432
SN2008bc.json
0.015067397898713919
SN2003iu.json
None
SN2004ef.json
0.031003287895856158
SN2006al.json
0.067849
SN2004bd.json
0.00893
SN2001ep.json
0.013012
SN2008ha.json
0.004623
SN1996ai.json
0.00316
SN2010jn.json
0.025047
SN2004bv.json
0.01061
PTF09dnp.json
0.037559
SN2005ag.json
0.079402
SN2003hu.json
None
PTF10qjq.json
0.0284
SN2006gr.json
0.034601
SN2003Y.json
0.01694
SN2008hu.json
0.05056
PTF10qsc.json
0.0879
SN1998cs.json
0.03256
SN2013aa.json
0.003999
SN2007sr.json
0.005417
SN2006ej.json
0.020458277727554597
SN2007gi.json
0.00482
LSQ14nr.json
0.091
SN2005hc.json
0.045948829040162176
SN2009ig.json
0.00885
SN1991B.json
0.008579
SN2007ba.json
0.038513
SN2006S.json
0.032102
SN2002hd.json
0.035001
SN2001en.json
0.015871
SN20

/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


SN2010gn.json
0.0365
SN2006ob.json
0.059238
SN1996bo.json
0.01728
SN2007kk.json
0.041045
SN2000cx.json
0.007929
SN2001ex.json
0.02638
SN2002bf.json
0.024197
SN2002cs.json
0.01577
SN2008R.json
0.01350112193281583
PTF10vqv.json
redshift nao encontrado
SN2002cr.json
0.00948
SN2005al.json
0.012381930324267376
SN2003fd.json
0.060222
SN2002de.json
0.028116
SN1998bn.json
0.005981
SN2002fk.json
0.007132
SN2002er.json
0.008569
SN1994ae.json
0.004266
SN1999cw.json
0.01237
SN1998eg.json
0.02476
SN2006cp.json
0.022289
SN1999ek.json
0.01752
SN2014aa.json
0.016982
LSQ12ege.json
0.08
SN2000cn.json
0.02349
SN2009D.json
0.025007
SN2001fh.json
0.01299
SN1998de.json
0.01647
SN2012fr.json
0.0054
SN2007qe.json
0.0239
SN2005bl.json
0.02406
SN1999gh.json
0.007679
SN2005na.json
0.02629016098230008
SN2003ic.json
0.055672
SN2007A.json
0.017653315868428443
SN1995al.json
0.00514


In [10]:
save_path = '/home/joao/Documentos/PCA'

In [11]:
table = []
for l in range(0,len(maxvalues)):
    table.append((maxvalues[l][1], maxvalues[l][0]))

name_of_file = "max_valuesV"

completeName = os.path.join(save_path, name_of_file+".txt")

f = open(completeName, 'w')
f.write(tabulate(table, tablefmt="plain"))
f.close()